# Qiita記事のタイトルを自動生成するモデルの学習

事前学習済み日本語T5モデルを、今回の自動生成タスク用に転移学習（ファインチューニング）します。

- **T5（Text-to-Text Transfer Transformer）**: テキストを入力されるとテキストを出力するという統一的枠組みで様々な自然言語処理タスクを解く深層学習モデル（[解説](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html)）
- **事前学習**: 個別のタスク用に学習をする前に文法や一般的な言葉の意味を学習させること（教師なし学習や自己教師あり学習とWikipedia等の大規模データ（コーパス）を用いることで広く一般的な知識を持ったモデルを作れる）
- **転移学習、ファインチューニング**: 事前学習済みモデルを初期値にして、特定のタスク用に追加で学習を行うこと（主に教師あり学習）


今回は入出力が次の形式を持ったタスク用に転移学習します。
- **入力**: "body: {body}"をトークナイズしたトークンID列
- **出力**: "title: {title}"をトークナイズしたトークンID列

ここで、{body}は記事本文（を前処理したもの）、{title}はタイトルです。

### 補足

今回はタイトル自動生成タスク用のモデルを作りますが、データセット（TSVファイル）を他のタスク用のものに入れ替えれば、ほぼコードを変えることなく（最大入出力トークン数を指定するmax_input_lengthとmax_target_lengthは適切な値に変える必要があるでしょう）、他のタスク用のモデルを作ることもできます。ぜひ色々試してみてください。

## ライブラリやデータの準備

### パスワード設定

**※学習に用いるデータセットとモデルの解凍にはパスワードが必要です（一般には公開しません）。**  
**※次の変数 RESOURCES_PASSWORD に解凍用の秘密のパスワードを設定してから、以降の処理を実行してください。**

In [2]:
# データセットとモデルの解凍用パスワード
RESOURCES_PASSWORD = ""

### 依存ライブラリのインストール

In [3]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2
!pip install -q transformers==4.2.2 pytorch_lightning==1.2.1 sentencepiece

     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 7.0MB 50.9MB/s 
     |████████████████████████████████| 12.8MB 47.9MB/s 
     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 819kB 36.7MB/s 
     |████████████████████████████████| 1.2MB 51.4MB/s 
     |████████████████████████████████| 2.9MB 40.9MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
     |████████████████████████████████| 829kB 48.0MB/s 
     |████████████████████████████████| 112kB 57.3MB/s 
     |████████████████████████████████| 276kB 55.4MB/s 
     |████████████████████████████████| 1.3MB 55.3MB/s 
     |████████████████████████████████| 296kB 49.3MB/s 
     |████████████████████████████████| 143kB 58.7MB/s 


### 学習に利用するデータセットと事前学習済みモデルをダウンロード

In [4]:
!wget -O resources.tar "https://www.floydhub.com/api/v1/resources/qPA7vt4m7e4zYLtNtbJACG?content=true&download=true"
!tar xvf resources.tar

--2021-03-16 13:08:31--  https://www.floydhub.com/api/v1/resources/qPA7vt4m7e4zYLtNtbJACG?content=true&download=true
Resolving www.floydhub.com (www.floydhub.com)... 172.67.72.144, 104.26.1.30, 104.26.0.30, ...
Connecting to www.floydhub.com (www.floydhub.com)|172.67.72.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘resources.tar’

resources.tar           [             <=>    ] 415.33M  25.1MB/s    in 18s     

2021-03-16 13:08:50 (22.5 MB/s) - ‘resources.tar’ saved [435509248]

./
./qiita_title_generation.zip


### データセットと事前学習済みモデルの解凍

In [5]:
!unzip -P {RESOURCES_PASSWORD} qiita_title_generation.zip

Archive:  qiita_title_generation.zip
   creating: qiita_title_generation/
   creating: qiita_title_generation/pretrained_model/
  inflating: qiita_title_generation/pretrained_model/config.json  
  inflating: qiita_title_generation/pretrained_model/spiece.model  
  inflating: qiita_title_generation/pretrained_model/pytorch_model.bin  
   creating: qiita_title_generation/dataset/
  inflating: qiita_title_generation/dataset/train.tsv  
  inflating: qiita_title_generation/dataset/dev.tsv  
  inflating: qiita_title_generation/dataset/test.tsv  


### 学習結果を格納するディレクトリの作成

- output: チェックポイントの出力先
- model: 学習済みモデルの出力先

In [6]:
!mkdir -p output model

## 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [7]:
import argparse
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AdamW,get_linear_schedule_with_warmup

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(12345)

In [8]:
# 事前学習済みモデルの置き場所
MODEL_PATH = "/content/qiita_title_generation/pretrained_model"

# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/qiita_title_generation/dataset",  # データセットのディレクトリ
    output_dir="/content/output",  # 学習ログやチェックポイントの出力先
    model_name_or_path=MODEL_PATH,
    tokenizer_name_or_path=MODEL_PATH,

    learning_rate=1e-3,  # 学習率
    adam_epsilon=1e-8,
    weight_decay=0.0,
    # warmup_steps=0,
    # gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=64,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=12345,
)


### TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{入力される文字列}\t{出力される文字列}"です。

In [9]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
    
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def _build(self):
        # ファイルからデータを読み込む
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                inputs = [line[0]]
                targets = [line[1]]

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    inputs, max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    targets, max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)

試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [10]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], 
                           "test.tsv", 
                           input_max_len=512, target_max_len=64)

テストデータの1レコード目をみてみます。

In [11]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
body: pandasによるデータ処理メモ pandasに関する情報ページはわりと豊富なので、主にリンクのまとめです。 #実行環境 実行環境は、jupyter(ipython)notebookを使うのが良いと思います。 [windowsにpython3とjupyter notebook(旧:ipython notebook)をインストールする-qiita]( #pandasのインストールとインポート #dataframeの作成 ##データの新規作成 pd.dataframeでdataframeの作成ができる。注意点として、各列ごとのデータ数は一致している必要がある。 ##既存データの読み込み ###csv,tsvを開く pandasでcsv/tsvファイルの読み書き|mwsoft pandasでカラムサイズが一定でないcsv/tsvを読み込む:mwsoft blog pythonコーディング備忘録〜その3〜(pandasのread_csvを使いこなす)-自調自考の旅 #データを抽出する python pandasデータ選択処理をちょっと詳しく<前編>-statsfragments python pandasデータ選択処理をちょっと詳しく<中編>-statsfragments python pandasデータ選択処理をちょっと詳しく<後編>-statsfragments pandasでデータフレームを条件指定で参照する-qiita ##列(column)抽出 ###ラベル名から抽出カラムの指定 ##行(row)抽出 ###条件を指定して抽出 ###正規表現での検索による抽出 python pandas:正規表現を使いdataframeを検索-qiita <python,pandas>データフレーム文字列検索-ねこゆきのメモ ###欠測値(nan)の除去 #dataframeの結合 python pandas図でみるデータ連結/結合処理-statsfragments merge,join,and concatenateーpandas 0.18.1 documentation #dataframeの加工 ##データのソート ###数値でのソート pandas.dataframe.sort_valuesー</s>

B. 入力データ（Aの

### 学習処理クラス

[PyTorch-lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [12]:
class T5FineTuner(pl.LightningModule):

    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                             if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                             if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)

        # scheduler = get_linear_schedule_with_warmup(
        #     optimizer, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=self.t_total
        # )

        # return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]
        return [optimizer]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                            type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                            type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            # self.t_total = (
            #     (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            #     // self.hparams.gradient_accumulation_steps
            #     * float(self.hparams.num_train_epochs)
            # )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)


## 転移学習を実行

In [13]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  512,  # 入力文の最大トークン数 (<512)
    "max_target_length": 64,  # 出力文の最大トークン数 (<512)
    "train_batch_size":  8,
    "eval_batch_size":   8,
    "num_train_epochs":  4,  # 適切なエポック数にしてください。2～10くらいのはず。
    })
args = argparse.Namespace(**args_dict)

# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#     args.output_dir, prefix="checkpoint", 
#     monitor="val_loss", mode="min", save_top_k=1
# )

train_params = dict(
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    # accumulate_grad_batches=args.gradient_accumulation_steps,
    # checkpoint_callback=checkpoint_callback,
)

In [14]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


1

## 学習済みモデルの保存

※/content/modelディレクトリは、このColabインスタンスがリセットされたら削除されます。  
削除されない場所に保存したいのであれば、Google Driveに保存するのが簡単です。

ColabからGoogle Driveを利用する方法: https://qiita.com/kado_u/items/45b76f9a6f920bf0f786

In [15]:
# 学習済みモデル（とトークナイザー）を保存する
model_dir_name = "/content/model/"
model.model.save_pretrained(model_dir_name)
tokenizer.save_pretrained(model_dir_name)

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/spiece.model',
 '/content/model/added_tokens.json')

In [16]:
# もし、Google Driveに学習済みモデルを保存する場合は次の処理を実行
# ※Googleアカウントの認証が必要です。

# from google.colab import drive
# drive.mount('/content/drive')

# !mkdir -p /content/drive/MyDrive/qiita_title_generation_model

# model_dir_name = "/content/drive/MyDrive/qiita_title_generation_model"
# tokenizer.save_pretrained(model_dir_name)
# model.model.save_pretrained(model_dir_name)

## 学習済みモデルの読み込み

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# 学習済みモデルが格納されたディレクトリ
model_dir_name = "/content/model/"
# model_dir_name = "/content/drive/qiita_title_generation_model"

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(model_dir_name, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_name)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

## 全テストデータに対してタイトル生成を実行

In [18]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], 
                          "test.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=4, num_workers=4)

# 推論モード
trained_model.eval()

outputs = []
targets = []
for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    outs = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        return_dict_in_generate=True,
        output_scores=True,
        repetition_penalty=1.5,
        )

    dec = [tokenizer.decode(ids, skip_special_tokens=True, 
                          clean_up_tokenization_spaces=False) 
                        for ids in outs.sequences]
    target = [tokenizer.decode(ids, skip_special_tokens=True, 
                             clean_up_tokenization_spaces=False) 
                        for ids in batch["target_ids"]]

    outputs.extend(dec)
    targets.extend(target)


In [19]:
def postprocess_title(title):
  return re.sub(r"^title: ", "", title)

generatedが自動生成されたタイトル、actualが実際に人が付けたタイトルです。

In [20]:
import re

for output, target in list(zip(outputs, targets)):
  print(f"generated: {postprocess_title(output)}")
  print(f"actual:    {postprocess_title(target)}")
  print()

generated: python pandasデータ処理メモ
actual:    [python]pandasの使い方まとめ

generated: 良いプルリクエストを書くには
actual:    github「完璧なプルリクの書き方を教えるぜ」

generated: phpのinterfaceの命名規約を調べた
actual:    人生いろいろ、インターフェイス命名規則いろいろ

generated: vue.js+typescriptでのベストプラクティスについて考えてみた
actual:    vuexによる状態管理を含む最高に快適なvue.js+typescriptの開発環境を目指す話

generated: openai gym使い方メモ
actual:    openai gym入門

generated: mysqlの文字コードとデータベース/カラムの文字コード
actual:    mysqlで文字コードをutf8にセットする

generated: ai academyのpython文法速習編とpythonプログラミング入門編
actual:    【初心者向け】無料でpythonの基本文法を5時間で学ぼう!

generated: reactのエコシステムとその概要
actual:    2018年reactとreduxのエコシステム総まとめ

generated: createjsをはじめる人向け
actual:    createjsをはじめる人が知っておくとラクなこと

generated: android studio編
actual:    iosとの比較つき!androidでこんなアプリ,こんな機能を作りたかったらこれを見ろ!作りたいアプリに対応するクラス、ライブラリのまとめ!

generated: phpで一時ファイルを作る
actual:    phpで一時的なファイルポインタを扱う方法

generated: ossを毎日のように読まないでいる自分を救う方法
actual:    「ソースコード全部読まなきゃ病」と闘う方法

generated: dfs(深さ優先探索)超入門!〜グラフ理論の世界への入口〜【前編】
actual:    bfs(幅優先探索)超入門!〜キューを鮮やかに使いこなす〜

generated

## 以降は次のNotebookと同じ内容のため説明は省略

https://github.com/sonoisa/qiita-title-generation/blob/main/T5_ja_qiita_title_generation.ipynb

In [21]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [22]:
import re

CODE_PATTERN = re.compile(r"```.*?```", re.MULTILINE | re.DOTALL)
LINK_PATTERN = re.compile(r"!?\[([^\]\)]+)\]\([^\)]+\)")
IMG_PATTERN = re.compile(r"<img[^>]*>")
URL_PATTERN = re.compile(r"(http|ftp)s?://[^\s]+")
NEWLINES_PATTERN = re.compile(r"(\s*\n\s*)+")

def clean_markdown(markdown_text):
    markdown_text = CODE_PATTERN.sub(r"", markdown_text)
    markdown_text = LINK_PATTERN.sub(r"\1", markdown_text)
    markdown_text = IMG_PATTERN.sub(r"", markdown_text)
    markdown_text = URL_PATTERN.sub(r"", markdown_text)
    markdown_text = NEWLINES_PATTERN.sub(r"\n", markdown_text)
    markdown_text = markdown_text.replace("`", "")
    return markdown_text

def normalize_text(markdown_text):
    markdown_text = clean_markdown(markdown_text)
    markdown_text = markdown_text.replace("\t", " ")
    markdown_text = normalize_neologd(markdown_text).lower()
    markdown_text = markdown_text.replace("\n", " ")
    return markdown_text

def preprocess_qiita_body(markdown_text):
    return "body: " + normalize_text(markdown_text)[:4000]

In [23]:
qiita_body = """
AIの進歩はすごいですね。

今回は深層学習を用いて、記事（Qiita）のタイトルを自動生成してくれるAIさんを試作してみました。
この実験は自然言語処理について新人さんに教えるためのハンズオンネタを探索する一環で行ったものになります。

作ったAIは、Qiitaの記事本文（少し前処理したテキスト）を与えると、適したタイトル文字列を作文して返してくれるというものです。

なお、学習データは（2019年頃に）Qiitaの殿堂を入り口にして、評価の高い記事（いいねが50個以上）をスクレイピングしたものを使いました。
つまりヒットした記事のタイトルの付け方を学んだAIであるといえます。

* もう少し詳細:
  * 学習データの例:
    * 入力: "body: hiveqlではスピードに難を感じていたため、私もprestoを使い始めました。 mysqlやhiveで使っていたクエリ..."
    * 出力: "title: hadoop利用者ならきっと知ってる、hive/prestoクエリ関数の挙動の違い"
  * 学習方法: 独自に作った日本語T5の事前学習モデルをこの学習データを用いて転移学習

以下、結果（抜粋）です。generatedが生成されたもの、actualが人が付けたタイトルです。
"""

In [24]:
preprocess_qiita_body(qiita_body)

'body: aiの進歩はすごいですね。 今回は深層学習を用いて、記事(qiita)のタイトルを自動生成してくれるaiさんを試作してみました。 この実験は自然言語処理について新人さんに教えるためのハンズオンネタを探索する一環で行ったものになります。 作ったaiは、qiitaの記事本文(少し前処理したテキスト)を与えると、適したタイトル文字列を作文して返してくれるというものです。 なお、学習データは(2019年頃に)qiitaの殿堂を入り口にして、評価の高い記事(いいねが50個以上)をスクレイピングしたものを使いました。 つまりヒットした記事のタイトルの付け方を学んだaiであるといえます。 *もう少し詳細: *学習データの例: *入力:\xa0"body:\xa0hiveqlではスピードに難を感じていたため、私もprestoを使い始めました。\xa0mysqlやhiveで使っていたクエリ..." *出力:\xa0"title:\xa0hadoop利用者ならきっと知ってる、hive/prestoクエリ関数の挙動の違い" *学習方法:\xa0独自に作った日本語t5の事前学習モデルをこの学習データを用いて転移学習 以下、結果(抜粋)です。generatedが生成されたもの、actualが人が付けたタイトルです。'

In [25]:
MAX_SOURCE_LENGTH = 512  # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = 64   # 生成されるタイトルの最大トークン数

# 推論モード設定
trained_model.eval()

# 前処理とトークナイズを行う
inputs = [preprocess_qiita_body(qiita_body)]
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
    padding="longest", return_tensors="pt")

input_ids = batch['input_ids']
input_mask = batch['attention_mask']
if USE_GPU:
  input_ids = input_ids.cuda()
  input_mask = input_mask.cuda()

# 生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask, 
    max_length=MAX_TARGET_LENGTH,
    return_dict_in_generate=True, output_scores=True,
    temperature=1.0,            # 生成にランダム性を入れる温度パラメータ
    num_beams=10,               # ビームサーチの探索幅
    diversity_penalty=1.0,      # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,         # ビームサーチのグループ数
    num_return_sequences=10,    # 生成する文の数
    repetition_penalty=1.5,     # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークン列を文字列に変換する
generated_titles = [tokenizer.decode(ids, skip_special_tokens=True, 
                                     clean_up_tokenization_spaces=False) 
                    for ids in outputs.sequences]

# 生成されたタイトルを表示する
for i, title in enumerate(generated_titles):
  print(f"{i+1:2}. {postprocess_title(title)}")

 1. 深層学習でqiitaのタイトルを自動生成してくれるaiを試作した
 2. aiの進化はすごい。記事タイトル自動生成のaiを試作した
 3. ディープラーニングで記事のタイトルを自動生成してくれるaiを試作した
 4. ディープラーニングで記事のタイトルを自動生成してくれるaiを試作した。
 5. aiがすごい勢いで記事のタイトルを自動生成するaiを試作した
 6. 【人工知能】深層学習で「記事タイトルを自動生成」する
 7. deep learningでqiitaのタイトルを自動生成するaiを作ってみた
 8. deep learningでqiitaのタイトルを自動生成するaiを作ってみた。
 9. 「記事のタイトルを自動生成してくれるai」を作ってみた。(結果)
10. 「記事のタイトルを自動生成してくれるai」を作ってみた
